In [26]:
import pandas as pd
import spacy
from collections import Counter
import re 
import pandas as pd 
import os
import numpy as np
import sys
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import GridSearchCV


from sklearn.feature_extraction.text import TfidfVectorizer
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

from sklearn.metrics import classification_report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Laden Preprocess dateien

In [27]:
data = pd.read_csv("preprocessed/dataLower.csv", on_bad_lines='skip', sep=';')
data = data.iloc[:,1:3]

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed/dataLower.csv'

### 20newsgroup.csv Datei einlesen und bereinigen nach Index, Duplikaten und Na
### Optional (Falls keine PreProcess verwendet)


In [28]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data = data.dropna()
data = data.drop_duplicates()
data = data.iloc[:,1:3]

### Aufteilung data nach Zielklassen

In [29]:
# Dictionary für die Gruppen-DataFrames
dataAth = data.drop(data[data["group"] != 0 ].index)
dataGraphics = data.drop(data[data["group"] != 1].index)
dataSpace = data.drop(data[data["group"] != 2].index)
dataReli = data.drop(data[data["group"] != 3 ].index)
grouped_data = [dataAth, dataGraphics, dataSpace, dataReli]


### Filtern nach Nomen, nur Alpha u. lemmatisierte Wörter
#### Optional (Falls keine PreProcess verwendet)

Erkentnisse:
- Wenn TFIDF anhand von Nomen ermittelt, erhält man mehrere Wörter bei dem gleichen threshold

In [ ]:
# Lade das spaCy-Modell
nlp = spacy.load('en_core_web_sm')

# Funktion zur Vorverarbeitung des Textes
def preprocess_only_nouns(text):
    # Text vorverarbeiten: In Kleinbuchstaben umwandeln
    text = text.lower()
    
    # Wende spaCy NLP-Pipeline auf den Text an
    doc = nlp(text)
    
    # Extrahiere nur Nomen (Substantive), lemmatisiere sie und entferne Stoppwörter
    lemmatized_tokens = [
        token.lemma_ for token in doc 
        if token.pos_ == 'NOUN' and not token.is_stop and token.is_alpha
    ]
    
    # Füge die Tokens wieder zu einem Text zusammen
    return ' '.join(lemmatized_tokens)

data['text'] = data['text'].apply(preprocess_only_nouns)
dataGraphics = dataGraphics['text'].apply(preprocess_only_nouns)
dataSpace = dataSpace['text'].apply(preprocess_only_nouns)
dataAth = dataAth['text'].apply(preprocess_only_nouns)
dataReli = dataReli['text'].apply(preprocess_only_nouns)



### Liste Wörter welche einen bestimmtem threshold haben erzeugen (für Weiterverarbietung)

Problematik:
- Diee max_tfidf_scores enhält jeweils den maximalen TF-IDF-Wert eines Wortes spiegelt nur seine Relevanz in einem einzigen Dokument wider, ohne zu berücksichtigen, ob das Wort in anderen Dokumenten relevant ist oder zur Klassifikation beiträgt.
=> Lösungs IDEE: Klassen trennen und den Durchschnitsswert nehmen, von den Documenten in denen der TFIDF-Wert nicht null ist

In [6]:
threshold = 0.4

# Berechne die TF-IDF-Werte
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['text'])

# Hole die Wortliste (Feature-Namen), keine duplikate
features = vectorizer.get_feature_names_out()
tfidf_matrix_df = pd.DataFrame(tfidf_matrix.toarray(), columns=features)

# Extrahiere die maximalen TF-IDF-Werte für jedes Wort
max_tfidf_scores = tfidf_matrix.max(axis=0).toarray().flatten()

# Filtere Wörter, deren TF-IDF-Wert den Schwellenwert überschreiten
tfidf_relevant_words = [features[i] for i in range(len(features)) if max_tfidf_scores[i] > threshold]

# Gib die relevanten Wörter aus
tfidf_relevant_words = list(set(tfidf_relevant_words))
print("Relevante Wörter:\n", tfidf_relevant_words)
print("\nLänge:\n", len(tfidf_relevant_words))
print("\ntfidf_scores:\n", max_tfidf_scores)


Relevante Wörter:
 ['rgb', 'afghans', 'sunlight', 'muller', 'sigkids', 'door', 'imagine', 'nudists', 'gifs', 'groothuis', 'concrete', 'year', 'vms', 'keith', 'increase', 'postscript', 'cx5', 'v2', 'acceleration', 'dock', 'tagreed', 'rayshade', 'spinoffs', 'atm', 'confused', '_email_', 'with', 'eternal', 'map', 'naplps', 'cockroaches', 'planets', 'islam', 'quote', 'van', 'dm', 'iff', '_rejects_', 'ra', 'image', 'mars', 'auggestions', 'hga', 'tnot', 'rendering', 'arm', 'tshe', 'weitek', 'sounds', 'commandment', 'p3', 'ocean', 'bullshit', 'bet', 'amongst', 'leather', 'tolerances', 'mpeg_play', 'deseases', 'radiosity', 'ramjet', 'hasn', 'twhy', 'blood', 'malcolm', '_simple', 'hacker', 'pens', 'pub', 'thanks', 'baptize', 'vmode', 'curve', 'penalty', 'sherzer', 'heliocentric', 'special', 'comparable', 'rastorle', 'tiger', 'incestuous', 'card', 'gps', 'temperature', 'angular', 'called', '________________________________________________________________________', 'jupiter', 'star', 'raoul', 'ta

Test unterschied Ergebnis relevanteste wörter und relevanteste Nomen vergleich F Wert

### Speichrn TFIDF-Matrix in csv

In [11]:
tfidf_matrix_df.to_csv("tfidf_matrix.csv", index=False)
print("TF-IDF-Matrix wurde gespeichert.")
tfidf_matrix_df.head

KeyboardInterrupt: 

### Liste n-Wörter pro Kategorie mit dem höchsten tfidf Wert erzeugen (für Weiterverarbietung)

Weitere Ansätze 
- Ansatz mit avg_tfidf_scores ausprobieren

In [ ]:
def get_top_n_words_per_category(n=400):
    # Initialisiere den TfidfVectorizer
    vectorizer = TfidfVectorizer(min_df=1)

    # Liste um die Top n Wörter zu speichern
    top_words_total = []
    top_words_by_group = {}

    for group_data in grouped_data:


        group_tfidf_matrix = vectorizer.fit_transform(group_data['text']) #TODO nicht so effezient


        # Hole die Wortliste (Feature-Namen)
        features = vectorizer.get_feature_names_out()
        
        # TODO OPT Berechne den Durchschnittlichen TF-IDF-Wert pro Wort für diese Kategorie, ggfs erweitern mit rausfiltern der Nullen 
        # avg_tfidf_scores = group_tfidf_matrix.mean(axis=0).A.flatten() #TODO: Testen mit 0 Zeilen da eigentlich gute Kombination aus tfidf und count => schlecht nur Wörter wie the etc.


        # Extrahiere die maximalen TF-IDF-Werte für jedes Wort
        # max_tfidf_scores = group_tfidf_matrix.max(axis=0).toarray().flatten()
        print(max_tfidf_scores)
        
        # Extrahiere die Wörter mit den höchsten TF-IDF-Werten (Top n)
        top_n_indices = max_tfidf_scores.argsort()[-n:][::-1]  # Die Indices der n höchsten Scores
        
        top_words_group = [features[i] for i in top_n_indices]
        top_words_by_group[group_data['group'].iloc[0]] = top_words_group

        # Speichere die Top n Wörter in der Liste
        top_words_total.extend(top_words_group)
    
    # Füge alle relevanten Wörter zusammen (optional, falls du sie vereinen möchtest)
    top_words_total = list(set(top_words_total))

    return top_words_by_group, top_words_total


# Aufruf der Funktion
top_words_by_group, top_words_total = get_top_n_words_per_category()

# Ausgabe der Top-Wörter pro Gruppe
print("Top TF-IDF-Wörter pro Gruppe:")
for group, words in top_words_by_group.items():
    print(f"Gruppe {group}: {', '.join(words[:10])} ...")  # Ausgabe der ersten 10 Wörter pro Gruppe

# Ausgabe aller einzigartigen Top-Wörter
print("\nAlle einzigartigen Top-Wörter (gesamt):")
print(", ".join(top_words_total))
print(f"Anzahl aller einzigartigen Wörter: {len(top_words_total)}")

[2.07430544e-05 1.79506788e-03 1.40954830e-04 ... 1.33462355e-04
 5.26707546e-05 2.83761287e-04]
[2.79997697e-03 1.04647523e-03 7.62626796e-06 ... 1.05078985e-05
 4.00014567e-04 2.10241626e-04]
[2.08297565e-03 2.83225457e-03 2.03942853e-04 ... 1.78051008e-05
 1.78051008e-05 3.56102017e-05]
[7.00768892e-05 8.76285921e-04 1.40709352e-04 ... 1.43762817e-05
 2.12377260e-04 4.02297981e-05]
Top TF-IDF-Wörter pro Gruppe:
Gruppe 0: the, of, to, is, that, and, you, it, in, not ...
Gruppe 1: the, to, of, and, it, is, for, in, you, that ...
Gruppe 2: the, to, of, and, is, in, it, that, you, for ...
Gruppe 3: the, to, of, that, and, is, you, in, it, not ...

Alle einzigartigen Top-Wörter (gesamt):
vga, dxf, off, this, video, through, line, statements, polygons, directory, development, rushdie, same, package, jim, word, that, islamic, algorithms, cause, technology, send, modes, see, help, does, even, moon, display, says, control, thus, care, world, again, gov, shareware, buy, orbital, have, working

## Tree Classifier mit Vektor trainieren

TODOs:
- Ggfs. autotrainer für TFID werte schreiben, bzw. wie ermittle ich die bessten werte
- Ggfs. Prüfen wie bei Vektoren Klassen einzeln behandeln
- Weitere Ansätze mit Lower etc.u. Nomen, ggfs einfach mit den Dateien von Moritz 
- Trennung Vectoren nach Group (siehe Chat gpt)
- Weitere Parameter testen
    tfidf = TfidfVectorizer(
    max_df=0.8,               # Häufige Wörter, die mehr als in 80% der Dokumente vorkommen, werden ignoriert
    min_df=2,                 # Nur Wörter, die in mindestens 2 Dokumenten vorkommen
    ngram_range=(1, 2),       # Unigramme und Bigramme
    stop_words='english',     # Entfernt englische Stoppwörter
    max_features=5000,        # Maximale Anzahl von Features
    use_idf=True,             # Verwende die inverse Dokumenthäufigkeit
    sublinear_tf=True         # Logarithmische Transformation der Termfrequenzen
    )


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Daten aufteilen
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['group'], test_size=0.2, random_state=42, stratify=data['group']
)

# Pipeline definieren
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=0.8,
        min_df=2,
        ngram_range=(1, 2),
        stop_words='english',
        max_features=5000
    )),
    ('clf', DecisionTreeClassifier(random_state=42))
])

# Parameter-Suchraum definieren
param_grid = {
    'tfidf__max_df': [0.4, 0.5, 0.6 ],          # Reduziere auf zwei sinnvolle Werte
    'tfidf__min_df': [1,3],             # Wesentlich: 1 (selten), 10 (mittlere Häufigkeit)
    'tfidf__max_features': [500, 5000],   # Fokus auf kleinere Werte für Geschwindigkeit
    'clf__max_depth': [70, 100, 130],    # Relevante Werte für Entscheidungsbaumtiefe
    'clf__min_samples_split': [1, 2, 4]     # Kleinere, repräsentative Auswahl
}

# GridSearchCV mit F1-Score als Bewertungsmaß
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,                          # 5-fache Cross-Validation
    scoring='f1_macro',            # Optimierung auf F1-Score (macro)
    n_jobs=-1                      # Parallele Verarbeitung
)

# Training mit Hyperparameter-Optimierung
grid_search.fit(X_train, y_train)

# Beste Parameter und Modell anzeigen
best_params = grid_search.best_params_
print("Beste Parameter:", best_params)

# Finales Modell aus GridSearchCV
best_pipeline = grid_search.best_estimator_

# Vorhersagen auf Testdaten
y_pred = best_pipeline.predict(X_test)

# Bewertung der Ergebnisse
print("F1-Score: ", f1_score(y_test, y_pred, average='macro'))
print("Precision: ", precision_score(y_test, y_pred, average='macro'))
print("Recall: ", recall_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

/home/dominik/vs-workspace/WiSe24/KI/projekt-git/KI-Python/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dominik/vs-workspace/WiSe24/KI/projekt-git/KI-Python/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dominik/vs-workspace/WiSe24/KI/projekt-git/KI-Python/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^

Beste Parameter: {'clf__max_depth': 130, 'clf__min_samples_split': 4, 'tfidf__max_df': 0.4, 'tfidf__max_features': 5000, 'tfidf__min_df': 1}
F1-Score:  0.6344803661734597
Precision:  0.6390127402416347
Recall:  0.6324285453703574
              precision    recall  f1-score   support

           0       0.53      0.52      0.52       155
           1       0.80      0.76      0.78       190
           2       0.65      0.73      0.69       190
           3       0.58      0.51      0.54       121

    accuracy                           0.65       656
   macro avg       0.64      0.63      0.63       656
weighted avg       0.65      0.65      0.65       656



Getrennte Vektorisierung

In [25]:
from scipy.sparse import hstack

# Kategorien im Datensatz
categories = data['group'].unique()

# Speichere die Vektorizer und die transformierten Daten
tfidf_per_category = {}
transformed_features = []

max_features = 1000  # Feste Anzahl der Features für alle Kategorien

# Trainiere für jede Kategorie einen separaten TF-IDF-Vektorizer
for category in categories:
    # Dokumente für die Kategorie filtern
    docs_in_category = data[data['group'] == category]['text']
    
    # TF-IDF für die Kategorie trainieren
    tfidf = TfidfVectorizer(max_df=0.9, min_df=1, stop_words='english', max_features=max_features)
    transformed = tfidf.fit_transform(docs_in_category)
    
    # Speichern des TF-IDF-Modells pro Kategorie
    tfidf_per_category[category] = tfidf
    transformed_features.append(transformed)

# Kombinieren der Features (Sparse-Matrix)
combined_features = hstack(transformed_features)

# Splitte die Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(combined_features, data['group'], test_size=0.2, random_state=42, stratify=data['group'])

# Klassifikator (z.B. DecisionTreeClassifier)
clf = DecisionTreeClassifier(max_depth=100, random_state=42)
clf.fit(X_train, y_train)

# Vorhersagen auf den Testdaten
y_pred = clf.predict(X_test)

# Bewertung des Modells
print(classification_report(y_test, y_pred))

ValueError: Mismatching dimensions along axis 0: {952, 776, 948, 604}